# Exploring the Graph of Dependencies

A small exploration of the graph of dependencies to understand the data and the structure of the graph.

## Data

In [1]:
import polars as pl

_ = pl.Config.set_tbl_rows(20)
_ = pl.Config.set_fmt_str_lengths(100)

In [4]:
import json
import urllib.request

url = "https://raw.githubusercontent.com/deepfunding/dependency-graph/refs/heads/main/graph/unweighted_graph.json"
with urllib.request.urlopen(url) as response:
    graph_data = json.loads(response.read())

df = pl.DataFrame(graph_data.get("links"))

# Drop the weight column
df = df.drop("weight")

rows, cols = df.shape
print(f"Rows: {rows}")
print(f"Columns: {cols}")

# Get unique sources and targets
sources = df.get_column("source").unique()
targets = df.get_column("target").unique()

print("Unique sources:", len(sources))
print("Unique targets:", len(targets))

df.sample(10)


Rows: 9896
Columns: 3
Unique sources: 18
Unique targets: 4289


relation,source,target
str,str,str
"""NPM""","""https://github.com/web3/web3.js""","""https://github.com/naugtur/xhr"""
"""NPM""","""https://github.com/web3/web3.js""","""https://github.com/metamask/json-rpc-engine"""
"""NPM""","""https://github.com/chainsafe/lodestar""","""https://github.com/sindresorhus/matcher"""
"""NPM""","""https://github.com/chainsafe/lodestar""","""https://github.com/mafintosh/generate-object-property"""
"""NPM""","""https://github.com/eth-infinitism/account-abstraction""","""https://github.com/sindresorhus/os-homedir"""
"""NPM""","""https://github.com/ethereum/remix-project""","""https://github.com/ljharb/call-bind"""
"""NPM""","""https://github.com/consensys/teku""","""https://github.com/sindresorhus/locate-path"""
"""NPM""","""https://github.com/ethereum/remix-project""","""https://github.com/omgovich/colord"""
"""NPM""","""https://github.com/web3/web3.js""","""https://github.com/remarkablemark/style-to-object"""


In [6]:
df.filter(pl.col("source") == "https://github.com/ethereum/solidity")

relation,source,target
str,str,str
"""PIP""","""https://github.com/ethereum/solidity""","""https://github.com/taminomara/sphinx-a4doc"""
"""PIP""","""https://github.com/ethereum/solidity""","""https://github.com/sphinx-doc/sphinx"""
"""PIP""","""https://github.com/ethereum/solidity""","""https://github.com/readthedocs/sphinx_rtd_theme"""


In [8]:
df.group_by("relation").agg(pl.len()).sort("relation", descending=True)

relation,len
str,u32
"""RUST""",1349
"""PIP""",135
"""NPM""",7814
"""GO""",598


In [10]:
# Split source and target URLs into organization and repository components
df = df.with_columns([
    pl.col("source").str.extract(r"https://github.com/([^/]+)/([^/]+)$", 1).alias("source_organization"),
    pl.col("source").str.extract(r"https://github.com/([^/]+)/([^/]+)$", 2).alias("source_repository"),
    pl.col("target").str.extract(r"https://github.com/([^/]+)/([^/]+)$", 1).alias("target_organization"),
    pl.col("target").str.extract(r"https://github.com/([^/]+)/([^/]+)$", 2).alias("target_repository")
])

df.sample(10)

relation,source,target,source_organization,source_repository,target_organization,target_repository
str,str,str,str,str,str,str
"""GO""","""https://github.com/prysmaticlabs/prysm""","""https://github.com/marten-seemann/tcp""","""prysmaticlabs""","""prysm""","""marten-seemann""","""tcp"""
"""RUST""","""https://github.com/grandinetech/grandine""","""https://github.com/strawlab/iana-time-zone""","""grandinetech""","""grandine""","""strawlab""","""iana-time-zone"""
"""NPM""","""https://github.com/web3/web3.js""","""https://github.com/grncdr/merge-stream""","""web3""","""web3.js""","""grncdr""","""merge-stream"""
"""NPM""","""https://github.com/ethereum/remix-project""","""https://github.com/mafintosh/pumpify""","""ethereum""","""remix-project""","""mafintosh""","""pumpify"""
"""NPM""","""https://github.com/ethereum/remix-project""","""https://github.com/maxogden/concat-stream""","""ethereum""","""remix-project""","""maxogden""","""concat-stream"""
"""GO""","""https://github.com/erigontech/erigon""","""https://github.com/pion/ice""","""erigontech""","""erigon""","""pion""","""ice"""
"""RUST""","""https://github.com/paradigmxyz/reth""","""https://github.com/rust-lang/regex""","""paradigmxyz""","""reth""","""rust-lang""","""regex"""
"""NPM""","""https://github.com/safe-global/safe-smart-account""","""https://github.com/mafintosh/mkdirp-classic""","""safe-global""","""safe-smart-account""","""mafintosh""","""mkdirp-classic"""
"""NPM""","""https://github.com/ethereum/remix-project""","""https://github.com/ajv-validator/ajv-formats""","""ethereum""","""remix-project""","""ajv-validator""","""ajv-formats"""


In [14]:
df.group_by("source").agg(pl.col("target").n_unique()).sort("target", descending=True)

source,target
str,u32
"""https://github.com/ethereum/remix-project""",2277
"""https://github.com/web3/web3.js""",1709
"""https://github.com/chainsafe/lodestar""",1514
"""https://github.com/eth-infinitism/account-abstraction""",854
"""https://github.com/ethereumjs/ethereumjs-monorepo""",796
"""https://github.com/safe-global/safe-smart-account""",519
"""https://github.com/paradigmxyz/reth""",463
"""https://github.com/sigp/lighthouse""",451
"""https://github.com/grandinetech/grandine""",435


In [16]:
df.group_by("target").agg(pl.col("source").n_unique()).sort("source", descending=True).head(10)

target,source
str,u32
"""https://github.com/sindresorhus/type-fest""",7
"""https://github.com/garycourt/uri-js""",7
"""https://github.com/isaacs/isexe""",7
"""https://github.com/sindresorhus/path-is-absolute""",7
"""https://github.com/sindresorhus/path-key""",7
"""https://github.com/sindresorhus/parent-module""",7
"""https://github.com/sindresorhus/string-width""",7
"""https://github.com/estools/esrecurse""",7
"""https://github.com/mathiasbynens/emoji-regex""",7


In [19]:
df.group_by("target_organization").agg(pl.col("target_repository").n_unique()).sort("target_repository", descending=True).head(10)

target_organization,target_repository
str,u32
"""sindresorhus""",253
"""jonschlinkert""",104
"""npm""",82
"""inspect-js""",49
"""syntax-tree""",46
"""isaacs""",44
"""ljharb""",43
"""gulpjs""",42
"""wooorm""",34


## Baseline Model

Let's work on a model that assigns the same weight to all dependencies. Source (`repo`) weights should sum to 1.

In [20]:
df = df.with_columns(
    (1 / pl.col("target").n_unique().over("source")).alias("weight")
)

In [21]:
df.select("source", "target", "weight").sample(10)

source,target,weight
str,str,f64
"""https://github.com/web3/web3.js""","""https://github.com/sindresorhus/read-pkg""",0.000585
"""https://github.com/ethereum/remix-project""","""https://github.com/jquery/esprima""",0.000439
"""https://github.com/prysmaticlabs/prysm""","""https://github.com/go-playground/universal-translator""",0.004098
"""https://github.com/safe-global/safe-smart-account""","""https://github.com/ndjson/ndjson.js""",0.001927
"""https://github.com/eth-infinitism/account-abstraction""","""https://github.com/sindresorhus/os-locale""",0.001171
"""https://github.com/safe-global/safe-smart-account""","""https://github.com/jonschlinkert/kind-of""",0.001927
"""https://github.com/chainsafe/lodestar""","""https://github.com/jshttp/forwarded""",0.000661
"""https://github.com/ethereumjs/ethereumjs-monorepo""","""https://github.com/mafintosh/why-is-node-running""",0.001256
"""https://github.com/web3/web3.js""","""https://github.com/euank/node-parse-numeric-range""",0.000585


In [22]:
df.group_by("source").agg(pl.col("weight").sum())

source,weight
str,f64
"""https://github.com/sigp/lighthouse""",1.0
"""https://github.com/ethereum/remix-project""",1.0
"""https://github.com/ethereum/web3.py""",1.0
"""https://github.com/eth-infinitism/account-abstraction""",1.0
"""https://github.com/erigontech/erigon""",1.0
"""https://github.com/consensys/teku""",1.0
"""https://github.com/ethereumjs/ethereumjs-monorepo""",1.0
"""https://github.com/ethereum/py-evm""",1.0
"""https://github.com/ethereum/go-ethereum""",1.0


## Heuristic Model

Now, lets assign weights based on external data. In this case, we will use the number of stars on GitHub for each package. Data can be obtained from GitHub API, [GitHub Archive](https://www.gharchive.org/), or, my prefered way, using [Open Source Observer public datasets on BigQuery](https://www.opensource.observer/).

In [23]:
import polars as pl
from google.cloud import bigquery

client = bigquery.Client()

In [38]:
package_list_string = df.select(pl.col("target")).to_series().str.join("', '")

In [59]:
QUERY = f"""
select
  artifact_url as target,
  is_fork,
  star_count,
  fork_count,
  created_at,
  updated_at
from `opensource-observer.oso.repositories_v0`
where artifact_url in ('{package_list_string[0]}')
"""

query_job = client.query(QUERY)
rows = query_job.result()

oso_df = pl.from_arrow(rows.to_arrow())

print(f"Rows: {oso_df.shape[0]}")

oso_df.head()

Rows: 1439


target,is_fork,star_count,fork_count,created_at,updated_at
str,bool,i64,i64,"datetime[μs, UTC]","datetime[μs, UTC]"
"""https://github.com/npm/agent""",false,7,5,2022-06-30 17:47:07 UTC,2024-09-06 23:49:51 UTC
"""https://github.com/npm/security-holder""",false,157,58,2016-03-23 00:22:33 UTC,2024-12-11 17:50:51 UTC
"""https://github.com/jsdom/domexception""",false,21,8,2017-08-14 03:04:01 UTC,2024-05-11 21:05:28 UTC
"""https://github.com/jsdom/abab""",false,93,19,2015-08-29 03:21:16 UTC,2024-05-17 16:43:47 UTC
"""https://github.com/herumi/mcl-wasm""",false,59,18,2017-11-20 06:46:44 UTC,2024-09-18 03:05:14 UTC


In [57]:
# Find targets in df that are not in oso_df
missing_targets = df.filter(~df["target"].is_in(oso_df["target"]))["target"].unique()
print(f"Number of missing targets: {len(missing_targets)}")
print(f"Number of targets in df: {len(df['target'].unique())}")
print(f"Number of targets in oso_df: {len(oso_df['target'].unique())}")
print(f"Percentage of missing targets: {(len(missing_targets) / len(df['target'].unique())) * 100:.2f}%")

missing_targets.sample(10)


Number of missing targets: 2850
Number of targets in df: 4289
Number of targets in oso_df: 1439
Percentage of missing targets: 66.45%


target
str
"""https://github.com/testing-library/dom-testing-library"""
"""https://github.com/octokit/endpoint.js"""
"""https://github.com/consensys/gnark-crypto"""
"""https://github.com/mafintosh/protocol-buffers-schema"""
"""https://github.com/nstepien/iltorb"""
"""https://github.com/jupyter/nbconvert"""
"""https://github.com/samthor/fast-text-encoding"""
"""https://github.com/mr-tron/base58"""
"""https://github.com/reem/rust-void"""
